In [30]:
#Cell 1 – Install & Imports
!pip install shap --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

import shap

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow:", tf.__version__)
import tensorflow as tf

# Make sure everything runs in eager mode
tf.config.run_functions_eagerly(True)
print("Eager mode:", tf.executing_eagerly())


TensorFlow: 2.19.0
Eager mode: False


In [33]:
#Cell 2 – Generate Multivariate Time Series
def generate_synthetic_multivariate_series(n_steps: int = 2000) -> pd.DataFrame:
    t = np.arange(n_steps)

    base_trend = 0.01 * t
    seasonal_1 = 2.0 * np.sin(2 * np.pi * t / 50)
    seasonal_2 = 1.5 * np.sin(2 * np.pi * t / 30)
    seasonal_3 = 1.0 * np.sin(2 * np.pi * t / 70)

    noise1 = np.random.normal(scale=0.5, size=n_steps)
    noise2 = np.random.normal(scale=0.3, size=n_steps)
    noise3 = np.random.normal(scale=0.2, size=n_steps)
    noise4 = np.random.normal(scale=0.7, size=n_steps)
    noise5 = np.random.normal(scale=0.5, size=n_steps)

    sensor_1 = base_trend + seasonal_1 + noise1
    sensor_2 = 0.5 * base_trend + seasonal_2 + noise2
    sensor_3 = -0.3 * base_trend + seasonal_3 + noise3
    sensor_4 = 0.1 * base_trend + 0.5 * seasonal_1 + noise4
    sensor_5 = 0.2 * base_trend - 0.3 * seasonal_2 + noise5

    target = np.roll(sensor_1, -1)

    data = {
        "sensor_1": sensor_1[:-1],
        "sensor_2": sensor_2[:-1],
        "sensor_3": sensor_3[:-1],
        "sensor_4": sensor_4[:-1],
        "sensor_5": sensor_5[:-1],
        "target":   target[:-1],
    }
    df = pd.DataFrame(data)
    return df

df = generate_synthetic_multivariate_series(2000)
print(df.head())
print("Shape:", df.shape)


   sensor_1  sensor_2  sensor_3  sensor_4  sensor_5    target
0  0.248357 -0.202553 -0.172699 -0.779857 -0.016513  0.191534
1  0.191534  0.273512  0.080399 -0.315318 -0.343385  0.841224
2  0.841224  0.382379  0.176160 -0.408752 -0.265219  1.527764
3  1.527764  0.804289  0.351563 -0.012473  0.098863  0.886431
4  0.886431  0.566633  0.066003  0.335848  0.312513  1.108502
Shape: (1999, 6)


In [34]:
#Cell 3 – Windowing & Metrics Helpers
def create_windows(features, targets, seq_len=30, horizon=1):
    X, y = [], []
    total_steps = len(features)
    for start in range(total_steps - seq_len - horizon + 1):
        end = start + seq_len
        X.append(features[start:end, :])
        y.append(targets[end + horizon - 1])
    return np.array(X), np.array(y)

def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100.0
    return mae, rmse, mape


In [35]:
#Cell 4 – Prepare Data (scale, split, windows)
feature_cols = ["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]
target_col = "target"

features = df[feature_cols].values
target = df[target_col].values

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

SEQ_LEN = 30
HORIZON = 1

X_all, y_all = create_windows(features_scaled, target, seq_len=SEQ_LEN, horizon=HORIZON)
print("X_all:", X_all.shape, "y_all:", y_all.shape)

n_samples = len(X_all)
train_end = int(0.7 * n_samples)
val_end   = int(0.85 * n_samples)

X_train, y_train = X_all[:train_end],      y_all[:train_end]
X_val,   y_val   = X_all[train_end:val_end], y_all[train_end:val_end]
X_test,  y_test  = X_all[val_end:],        y_all[val_end:]

print("Train:", X_train.shape)
print("Val  :", X_val.shape)
print("Test :", X_test.shape)


X_all: (1969, 30, 5) y_all: (1969,)
Train: (1378, 30, 5)
Val  : (295, 30, 5)
Test : (296, 30, 5)


In [41]:
#Cell 5 – Build Model Function
def build_lstm_model(input_shape,
                     num_layers=2,
                     hidden_size=64,
                     learning_rate=1e-3):
    # Clear any old graphs/models
    tf.keras.backend.clear_session()

    model = models.Sequential(name="LSTM_Forecaster")

    # Explicit Input layer
    model.add(layers.Input(shape=input_shape))

    for i in range(num_layers):
        return_sequences = (i < num_layers - 1)
        model.add(
            layers.LSTM(
                hidden_size,
                return_sequences=return_sequences,
                name=f"lstm_{i+1}",
            )
        )

    model.add(layers.Dense(1, name="output"))

    optimizer = optimizers.Adam(learning_rate=learning_rate)

    # ⭐ run_eagerly=True avoids the SymbolicTensor / Dataset issues
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["mae"],
        run_eagerly=True,
    )

    return model

input_shape = (SEQ_LEN, X_all.shape[2])


In [42]:
# Make sure we are in normal graph mode, not experimental eager mode
param_grid = {
    "num_layers":   [1, 2],
    "hidden_size":  [32, 64],
    "learning_rate":[1e-3, 3e-4],
    "batch_size":   [32, 64],
}

best_config = None
best_val_mae = np.inf
best_model = None

for nl in param_grid["num_layers"]:
    for hs in param_grid["hidden_size"]:
        for lr in param_grid["learning_rate"]:
            for bs in param_grid["batch_size"]:

                config = {
                    "num_layers": nl,
                    "hidden_size": hs,
                    "learning_rate": lr,
                    "batch_size": bs,
                }
                print("\nConfig:", config)

                model = build_lstm_model(
                    input_shape=input_shape,
                    num_layers=nl,
                    hidden_size=hs,
                    learning_rate=lr,
                )

                es = callbacks.EarlyStopping(
                    monitor="val_mae",
                    patience=5,
                    restore_best_weights=True,
                )

                history = model.fit(
                    X_train,
                    y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=bs,
                    callbacks=[es],
                    verbose=0,
                )

                val_mae = float(np.min(history.history["val_mae"]))
                print("Validation MAE:", val_mae)

                if val_mae < best_val_mae:
                    best_val_mae = val_mae
                    best_config = config
                    best_model = model

print("\nBest hyperparameters:", best_config)
print("Best validation MAE:", best_val_mae)




Config: {'num_layers': 1, 'hidden_size': 32, 'learning_rate': 0.001, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [38]:
#Cell 7 – Retrain Best Model on Train+Val and Evaluate
# Merge train + val
X_train_full = np.concatenate([X_train, X_val], axis=0)
y_train_full = np.concatenate([y_train, y_val], axis=0)

final_model = build_lstm_model(
    input_shape,
    num_layers=best_config["num_layers"],
    hidden_size=best_config["hidden_size"],
    learning_rate=best_config["learning_rate"],
)

es_final = callbacks.EarlyStopping(
    monitor="val_mae",
    patience=8,
    restore_best_weights=True
)

history_final = final_model.fit(
    X_train_full, y_train_full,
    validation_split=0.1,
    epochs=80,
    batch_size=best_config["batch_size"],
    callbacks=[es_final],
    verbose=1
)

y_pred = final_model.predict(X_test).flatten()

mae, rmse, mape = regression_metrics(y_test, y_pred)
print("\n===== Test Metrics =====")
print("MAE :", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

plt.figure()
plt.plot(y_test, label="True")
plt.plot(y_pred, label="Predicted")
plt.title("LSTM Forecast: True vs Predicted (Test)")
plt.xlabel("Time index")
plt.ylabel("Target")
plt.legend()
plt.tight_layout()
plt.show()


TypeError: 'NoneType' object is not subscriptable

In [39]:
#Cell 8 – SHAP Explainability
# Use a small subset for SHAP (to keep it fast)
background_size = min(200, len(X_train_full))
explain_size = min(100, len(X_test))

background = X_train_full[np.random.choice(len(X_train_full),
                                           size=background_size,
                                           replace=False)]
explain_samples = X_test[:explain_size]

explainer = shap.DeepExplainer(final_model, background)
shap_values_list = explainer.shap_values(explain_samples)
shap_values = shap_values_list[0]  # [samples, seq_len, num_features]

feature_names = feature_cols

# Global feature importance
feature_importance = np.mean(np.abs(shap_values), axis=(0, 1))

plt.figure()
plt.bar(range(len(feature_names)), feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title("Global Feature Importance (mean |SHAP| over time)")
plt.tight_layout()
plt.show()

# Time-step importance
timestep_importance = np.mean(np.abs(shap_values), axis=2)
mean_timestep_importance = np.mean(timestep_importance, axis=0)

plt.figure()
plt.plot(range(1, SEQ_LEN + 1), mean_timestep_importance)
plt.title("Average Time-Step Importance")
plt.xlabel("Time step in input window")
plt.ylabel("Mean |SHAP|")
plt.tight_layout()
plt.show()


NameError: name 'final_model' is not defined

In [27]:
# Refactor the code to temporarily toggle eager execution on and off, allowing Keras model training to run with eager execution enabled and SHAP explainability to run with eager execution disabled, addressing the `RuntimeError`.
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# NOTE: All tf.compat.v1.disable_eager_execution() and tf.compat.v1.disable_v2_behavior()
# calls are removed/commented out from the global scope and from inside main().
# TensorFlow will run in its default eager mode (TF2.x behavior).
# SHAP will be imported later and attempted to run in this eager environment.

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


# =========================================================
# 1. DATA GENERATION – multivariate time series (5 features)
# =========================================================

def generate_synthetic_multivariate_series(n_steps: int = 2000) -> pd.DataFrame:
    """
    Programmatically generate a multivariate time series that mimics
    industrial sensor data with:
      - clear trend
      - multiple seasonalities
      - Gaussian noise

    Returns:
        DataFrame with 5 sensor features and a 1-step-ahead target.
    """
    t = np.arange(n_steps)

    # Global trend
    base_trend = 0.01 * t

    # Different seasonal components
    seasonal_1 = 2.0 * np.sin(2 * np.pi * t / 50)   # period 50
    seasonal_2 = 1.5 * np.sin(2 * np.pi * t / 30)   # period 30
    seasonal_3 = 1.0 * np.sin(2 * np.pi * t / 70)   # period 70

    # Sensor-level noise
    noise1 = np.random.normal(scale=0.5, size=n_steps)
    noise2 = np.random.normal(scale=0.3, size=n_steps)
    noise3 = np.random.normal(scale=0.2, size=n_steps)
    noise4 = np.random.normal(scale=0.7, size=n_steps)
    noise5 = np.random.normal(scale=0.5, size=n_steps)

    # Five correlated sensor signals
    sensor_1 = base_trend + seasonal_1 + noise1
    sensor_2 = 0.5 * base_trend + seasonal_2 + noise2
    sensor_3 = -0.3 * base_trend + seasonal_3 + noise3
    sensor_4 = 0.1 * base_trend + 0.5 * seasonal_1 + noise4
    sensor_5 = 0.2 * base_trend - 0.3 * seasonal_2 + noise5

    # Target is one-step-ahead of sensor_1 (forecasting)
    target = np.roll(sensor_1, -1)

    # Drop last position (roll artifact)
    sensor_1 = sensor_1[:-1]
    sensor_2 = sensor_2[:-1]
    sensor_3 = sensor_3[:-1]
    sensor_4 = sensor_4[:-1]
    sensor_5 = sensor_5[:-1]
    target = target[:-1]

    df = pd.DataFrame(
        {
            "sensor_1": sensor_1,
            "sensor_2": sensor_2,
            "sensor_3": sensor_3,
            "sensor_4": sensor_4,
            "sensor_5": sensor_5,
            "target": target,
        }
    )
    return df


# =========================================================
# 2. SEQUENCE WINDOWING
# =========================================================

def create_windows(features: np.ndarray,
                   targets: np.ndarray,
                   seq_len: int = 30,
                   horizon: int = 1):
    """
    Convert multivariate time series into supervised learning windows.

    Args:
        features: array of shape [T, num_features]
        targets: array of shape [T,]
        seq_len: length of input sequence
        horizon: forecasting horizon (steps ahead)

    Returns:
        X: [num_samples, seq_len, num_features]
        y: [num_samples,]
    """
    X, y = [], []
    total_steps = len(features)
    for start in range(total_steps - seq_len - horizon + 1):
        end = start + seq_len
        X.append(features[start:end, :])
        y.append(targets[end + horizon - 1])
    return np.array(X), np.array(y)


# =========================================================
# 3. BUILD LSTM MODEL
# =========================================================

def build_lstm_model(input_shape,
                     num_layers: int = 2,
                     hidden_size: int = 64,
                     learning_rate: float = 1e-3) -> tf.keras.Model:
    """
    Build and compile an LSTM model for time series forecasting.

    Args:
        input_shape: (seq_len, num_features)
        num_layers: number of LSTM layers
        hidden_size: hidden units per layer
        learning_rate: learning rate for Adam optimizer
    """
    model = models.Sequential(name="LSTM_Forecaster")
    for i in range(num_layers):
        return_sequences = i < (num_layers - 1)
        if i == 0:
            model.add(
                layers.LSTM(
                    hidden_size,
                    return_sequences=return_sequences,
                    input_shape=input_shape,
                    name=f"lstm_{i+1}",
                )
            )
        else:
            model.add(
                layers.LSTM(
                    hidden_size,
                    return_sequences=return_sequences,
                    name=f"lstm_{i+1}",
                )
            )

    model.add(layers.Dense(1, name="output"))

    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer, metrics=["mae"])
    return model


# =========================================================
# 4. METRICS
# =========================================================

def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute MAE, RMSE, and MAPE for regression.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100.0
    return mae, rmse, mape


# =========================================================
# 5. FULL PIPELINE
# =========================================================

def main():
    # -----------------------------------------------------
    # 5.1 Dataset creation
    # -----------------------------------------------------
    df = generate_synthetic_multivariate_series(n_steps=2000)
    print("Dataset head:\n", df.head())
    print("Dataset shape:", df.shape)

    feature_cols = ["sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]
    target_col = "target"

    features = df[feature_cols].values
    target = df[target_col].values

    # Standardize features (target kept in original scale)
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    seq_len = 30
    horizon = 1

    X_all, y_all = create_windows(
        features_scaled,
        target,
        seq_len=seq_len,
        horizon=horizon,
    )
    print("X_all shape:", X_all.shape)  # (samples, seq_len, num_features)
    print("y_all shape:", y_all.shape)

    # -----------------------------------------------------
    # 5.2 Time-based train / val / test split
    # -----------------------------------------------------
    n_samples = len(X_all)
    train_end = int(0.7 * n_samples)
    val_end = int(0.85 * n_samples)

    X_train, y_train = X_all[:train_end], y_all[:train_end]
    X_val, y_val = X_all[train_end:val_end], y_all[train_end:val_end]
    X_test, y_test = X_all[val_end:], y_all[val_end:]

    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

    input_shape = (seq_len, X_all.shape[2])

    # -----------------------------------------------------
    # 5.3 Hyperparameter tuning – small grid search
    # -----------------------------------------------------
    param_grid = {
        "num_layers": [1, 2],
        "hidden_size": [32, 64],
        "learning_rate": [1e-3, 3e-4],
        "batch_size": [32, 64],
    }

    best_config = None
    best_val_mae = np.inf
    best_model = None

    for num_layers in param_grid["num_layers"]:
        for hidden_size in param_grid["hidden_size"]:
            for lr in param_grid["learning_rate"]:
                for batch_size in param_grid["batch_size"]:
                    config = {
                        "num_layers": num_layers,
                        "hidden_size": hidden_size,
                        "learning_rate": lr,
                        "batch_size": batch_size,
                    }
                    print("\nTraining configuration:", config)

                    model = build_lstm_model(
                        input_shape=input_shape,
                        num_layers=num_layers,
                        hidden_size=hidden_size,
                        learning_rate=lr,
                    )

                    es = callbacks.EarlyStopping(
                        monitor="val_mae",
                        patience=5,
                        restore_best_weights=True,
                    )

                    history = model.fit(
                        X_train,
                        y_train,
                        epochs=50,
                        batch_size=batch_size,
                        validation_data=(X_val, y_val),
                        callbacks=[es],
                        verbose=0,
                    )

                    val_mae = float(np.min(history.history["val_mae"]))
                    print(f"Validation MAE: {val_mae:.4f}")

                    if val_mae < best_val_mae:
                        best_val_mae = val_mae
                        best_config = config
                        best_model = model

    print("\nBest hyperparameters:", best_config)
    print("Best validation MAE:", best_val_mae)

    # -----------------------------------------------------
    # 5.4 Retrain best model on Train + Val
    # -----------------------------------------------------
    X_train_full = np.concatenate([X_train, X_val], axis=0)
    y_train_full = np.concatenate([y_train, y_val], axis=0)

    final_model = build_lstm_model(
        input_shape=input_shape,
        num_layers=best_config["num_layers"],
        hidden_size=best_config["hidden_size"],
        learning_rate=best_config["learning_rate"],
    )

    es_final = callbacks.EarlyStopping(
        monitor="val_mae",
        patience=8,
        restore_best_weights=True,
    )

    history_final = final_model.fit(
        X_train_full,
        y_train_full,
        epochs=80,
        batch_size=best_config["batch_size"],
        validation_split=0.1,
        callbacks=[es_final],
        verbose=1,
    )

    # -----------------------------------------------------
    # 5.5 Evaluation on test set
    # -----------------------------------------------------
    y_pred = final_model.predict(X_test).flatten()

    mae, rmse, mape = regression_metrics(y_test, y_pred)
    print("\n===== Test Metrics =====")
    print(f"MAE : {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")

    # Plot true vs predicted
    plt.figure()
    plt.plot(y_test, label="True")
    plt.plot(y_pred, label="Predicted")
    plt.title("LSTM Forecast: True vs Predicted (Test Set)")
    plt.xlabel("Time index")
    plt.ylabel("Target")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # -----------------------------------------------------
    # 5.6 Explainability with SHAP
    # -----------------------------------------------------
    # Import SHAP right before its usage to minimize potential TensorFlow eager conflicts
    import shap

    # SHAP DeepExplainer often requires graph mode (TensorFlow 1.x behavior).
    # Attempting to run SHAP in TF2.x eager mode. If this fails, then more advanced
    # techniques like a kernel restart or alternative explainability methods are needed.

    background_size = min(200, len(X_train_full))
    explain_size = min(200, len(X_test))

    background = X_train_full[
        np.random.choice(len(X_train_full), size=background_size, replace=False)
    ]
    explain_samples = X_test[:explain_size]

    # DeepExplainer for Keras model
    explainer = shap.DeepExplainer(final_model, background)
    shap_values_list = explainer.shap_values(explain_samples)
    # For regression models shap_values is a list with one element
    shap_values = shap_values_list[0]  # [samples, seq_len, num_features]

    # ---- Global feature importance over all time steps ----
    feature_importance = np.mean(np.abs(shap_values), axis=(0, 1))
    feature_names = feature_cols

    plt.figure()
    plt.bar(range(len(feature_names)), feature_importance)
    plt.xticks(range(len(feature_names)), feature_names, rotation=45)
    plt.title("Global Feature Importance (mean |SHAP| over time)")
    plt.tight_layout()
    plt.show()

    # ---- Time-step importance (which positions in sequence matter) ----
    timestep_importance = np.mean(np.abs(shap_values), axis=2)  # [samples, seq_len]
    mean_timestep_importance = np.mean(timestep_importance, axis=0)

    plt.figure()
    plt.plot(range(1, seq_len + 1), mean_timestep_importance)
    plt.title("Average Time-Step Importance (mean |SHAP| over features)")
    plt.xlabel("Time step in input window")
    plt.ylabel("Importance")
    plt.tight_layout()
    plt.show()

    # Optional SHAP summary plot (commented because it needs proper backend)
    # shap.summary_plot(
    #     shap_values.reshape(-1, shap_values.shape[-1]),
    #     explain_samples.reshape(-1, explain_samples.shape[-1]),
    #     feature_names=feature_names * seq_len,
    # )


if __name__ == "__main__":
    main()

Dataset head:
    sensor_1  sensor_2  sensor_3  sensor_4  sensor_5    target
0  0.248357 -0.202553 -0.172699 -0.779857 -0.016513  0.191534
1  0.191534  0.273512  0.080399 -0.315318 -0.343385  0.841224
2  0.841224  0.382379  0.176160 -0.408752 -0.265219  1.527764
3  1.527764  0.804289  0.351563 -0.012473  0.098863  0.886431
4  0.886431  0.566633  0.066003  0.335848  0.312513  1.108502
Dataset shape: (1999, 6)
X_all shape: (1969, 30, 5)
y_all shape: (1969,)
Train: (1378, 30, 5), Val: (295, 30, 5), Test: (296, 30, 5)

Training configuration: {'num_layers': 1, 'hidden_size': 32, 'learning_rate': 0.001, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/dataset_ops.py:2341: UserWarning: Seed 42 from outer graph might be getting used by function Dataset_map_permutation, if the random op has not been provided any seed. Explicitly set the seed in the function if this is not the intended behavior.
  return map_op._map_v2(


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.